In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class MaskedSelfAttention(nn.Module):
    def __init__(self, d_model = 2, row_dim = 0, col_dim = 1):
        # d_model : dimennsi dari model atau angka dari Word Embedding

        super().__init__()
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        # membuat matriks weight untuk query, key, dan value
        # nn.Linear juga akan melakukan operasi matematis yaitu y = xA^T + b
        # dimana x adalah input, A adalah weight, dan b adalah bias

        self.row_dim = row_dim
        self.col_dim = col_dim

    # operasi forward
    def forward(self, token_encodings, mask=None):
        # mask : None dimaksudkan jika melakuakn operasi original Self Attention
        # jika mask tidak none, maka melakukan operasi Masked Self Attention

        q = self.W_q(token_encodings)
        k = self.W_k(token_encodings)
        v = self.W_v(token_encodings)
        # mengalikan matriks weight dengan input

        ## MASKED SELF ATTENTION SCORE ## 

        # similiarity score
        sims = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim))

        # normalisasi similiarity score
        scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)

        # masking
        if mask is not None:
            # disini kita menggantikan nilai yang ingin kita masking
            # dengan nilai yang sangat kecil (negative infinity atau menkdekati 0)
            scaled_sims = scaled_sims.masked_fill(mask=mask, value=1e-9) # bisa diisi dengan -inf
            # dengan nilai yang mendekati 0
            # softmax akan mengembalikan 0

        # softmax
        attention_sm = F.softmax(scaled_sims, dim=self.col_dim)

        # kalikan dengan Value
        attention_score = torch.matmul(attention_sm, v)

        ## END ##
        
        return attention_score
